# Script for a period

In [33]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [34]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [35]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [36]:
data_giper

{'adverts': [{'type': 8,
   'status': 11,
   'count': 4,
   'advert_list': [{'advertId': 21526558,
     'changeTime': '2025-02-07T16:43:04.178341+03:00'},
    {'advertId': 22844037, 'changeTime': '2025-01-31T13:40:18.926964+03:00'},
    {'advertId': 22844946, 'changeTime': '2025-03-04T12:36:41.470052+03:00'},
    {'advertId': 23402220, 'changeTime': '2025-03-04T13:29:43.833312+03:00'}]},
  {'type': 9,
   'status': 11,
   'count': 5,
   'advert_list': [{'advertId': 22844853,
     'changeTime': '2025-02-27T14:07:48.633319+03:00'},
    {'advertId': 22844893, 'changeTime': '2025-01-31T13:31:32.654625+03:00'},
    {'advertId': 22950974, 'changeTime': '2025-03-03T12:35:20.701855+03:00'},
    {'advertId': 22951078, 'changeTime': '2025-03-03T12:47:32.279379+03:00'},
    {'advertId': 22951454, 'changeTime': '2025-03-03T15:33:24.073868+03:00'}]},
  {'type': 9,
   'status': 7,
   'count': 1,
   'advert_list': [{'advertId': 22954217,
     'changeTime': '2025-01-31T16:08:49.754418+03:00'}]},
  {'ty

In [37]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [38]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [39]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[8004530, 8743562, 9496209, 9496385, 9591336, 11296204, 11296333, 16255090, 16301721, 16301819, 16912359, 16912972, 16913380, 16913696, 16913774, 16914258, 16917733, 16917791, 16918232, 16918415, 16919369, 16920802, 16923161, 16923906, 16923933, 16924292, 17266442, 17563748, 17563873, 17563938, 17687135, 17687241, 17687293, 17687357, 17688586, 17688635, 18640342, 18870731, 18870828, 19262099, 19263710, 19263824, 19265149, 19265205, 19626785, 19662812, 19904919, 19994021, 20039654, 20139599], [20175678, 20334946, 20354793, 20397195, 21040251, 21040485, 21040635, 21040756, 21040856, 21040946, 21385613, 21386362, 21482495, 21482904, 21498202, 11296551, 12465028, 16144421, 20039743, 20955301, 20955474, 20955667, 20955944, 20956444, 21350648, 21351310, 21351587, 21385572, 21386314, 21820825, 22123202, 213920, 213946, 213954, 225156, 232543, 232547, 238682, 244654, 244667, 354788, 354832, 354853, 535702, 535721, 535733, 535812, 535864, 535918, 535940], [535

In [40]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-17,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [41]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-03-16",  # Replace with your start date
    "end": "2025-03-16"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [42]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [43]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [44]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [45]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [46]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,advertId,interval.begin,interval.end
0,7008,383,5.47,6.43,2462.83,37,9,2.35,9,28694,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21965489,2025-03-16,2025-03-16
1,1395,46,3.30,6.07,279.00,3,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21203170,2025-03-16,2025-03-16
2,2754,121,4.39,4.35,526.29,16,1,0.83,1,2709,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",22846531,2025-03-16,2025-03-16
3,1279,41,3.21,14.74,604.43,6,1,2.44,1,2839,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21883272,2025-03-16,2025-03-16
4,200,9,4.50,10.56,95.08,1,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21821199,2025-03-16,2025-03-16
5,583,28,4.80,4.79,134.09,2,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21212715,2025-03-16,2025-03-16
6,751,11,1.46,12.50,137.51,1,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",20124734,2025-03-16,2025-03-16
7,1417,84,5.93,3.66,307.24,8,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",19276746,2025-03-16,2025-03-16
8,1053,22,2.09,10.34,227.50,0,0,0.00,0,0,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21577713,2025-03-16,2025-03-16
9,3044,119,3.91,5.17,614.89,9,4,3.36,4,44915,"[{'date': '2025-03-16T03:00:00+03:00', 'views'...",21203007,2025-03-16,2025-03-16


In [47]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

45


In [48]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-16 03:00:00,159488075,Массажер для тела медицинский Nozomi MH-102 ин...,598,5,0.84,43.83,219.17,0,0,0.00,0,0,21965489
1,2025-03-16 03:00:00,159488075,Массажер для тела медицинский Nozomi MH-102 ин...,4825,283,5.87,5.89,1668.26,31,8,2.83,8,25573,21965489
2,2025-03-16 03:00:00,159488075,Массажер для тела медицинский Nozomi MH-102 ин...,1585,95,5.99,6.06,575.40,6,1,1.05,1,3121,21965489
3,2025-03-16 03:00:00,263723655,Бритва мужская электрическая S9 Pro 9519s с тр...,15,0,0.00,0.00,3.00,0,0,0.00,0,0,21203170
4,2025-03-16 03:00:00,263725116,Бритва мужская электрическая S8 8513s с триммером,13,0,0.00,0.00,2.60,0,0,0.00,0,0,21203170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,2025-03-16 03:00:00,203510173,Триммер для бороды и усов BT5420,216,9,4.17,3.32,29.84,1,0,0.00,0,0,18447862
665,2025-03-16 03:00:00,203510173,Триммер для бороды и усов BT5420,170,5,2.94,4.85,24.25,1,0,0.00,0,0,18447862
666,2025-03-16 03:00:00,224944973,Насадка для зубной щетки Cross Action 1 шт,281,10,3.56,4.02,40.21,0,0,0.00,0,0,16530753
667,2025-03-16 03:00:00,224944973,Насадка для зубной щетки Cross Action 1 шт,1512,82,5.42,2.66,217.72,16,3,3.66,3,1623,16530753


In [49]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_guten.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [50]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,23806943,2025-03-16 03:00:00,Вакууматор для продуктов с запайкой VC 11,415,10,1.986667,3.620000,66.21,0,0,0.000000,0,0,18870595,WB-GutenTech,Wildberries
1,25088481,2025-03-16 03:00:00,Эпилятор женский электрический Silk-epil 9-880,329,9,1.780000,11.390000,74.03,0,0,0.000000,0,0,21204285,WB-GutenTech,Wildberries
2,25623718,2025-03-16 03:00:00,Бритва мужская электрическая S3 300BT с триммером,107,5,3.416667,1.383333,9.93,1,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
3,25623718,2025-03-16 03:00:00,Бритва мужская электрическая S3 300BT с триммером,628,33,4.456667,6.446667,128.74,3,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
4,25624340,2025-03-16 03:00:00,Электробритва Series 3 300s,158,10,4.646667,0.820000,13.36,3,1,16.666667,1,5951,21202453,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,235490801,2025-03-16 03:00:00,Эпилятор Silk-epil 5 5-050,10,1,25.000000,0.060000,0.54,0,1,50.000000,1,6316,20822563,WB-Smart-Market,Wildberries
431,235490802,2025-03-16 03:00:00,Эпилятор S9 SES 9-030,2,0,0.000000,0.000000,0.10,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries
432,235599972,2025-03-16 03:00:00,Электробритва Series 9 9510s,528,31,5.893333,1.790000,55.92,3,0,0.000000,0,0,15562363,WB-Smart-Market,Wildberries
433,253299215,2025-03-16 03:00:00,Триммер для бороды и усов BT3421,310,20,4.886667,0.936667,32.18,3,1,3.703333,1,3225,19597465,WB-Smart-Market,Wildberries


In [51]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_grouped_combined_campaigns.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [52]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [53]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,23806943,2025-03-16 03:00:00,Вакууматор для продуктов с запайкой VC 11,415,10,66.21,0,0,0,0,18870595,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания
1,25088481,2025-03-16 03:00:00,Эпилятор женский электрический Silk-epil 9-880,329,9,74.03,0,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
2,25623718,2025-03-16 03:00:00,Бритва мужская электрическая S3 300BT с триммером,107,5,9.93,1,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25623718,2025-03-16 03:00:00,Бритва мужская электрическая S3 300BT с триммером,628,33,128.74,3,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
4,25624340,2025-03-16 03:00:00,Электробритва Series 3 300s,158,10,13.36,3,1,1,5951,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,235490801,2025-03-16 03:00:00,Эпилятор Silk-epil 5 5-050,10,1,0.54,0,1,1,6316,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-03-17,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
431,235490802,2025-03-16 03:00:00,Эпилятор S9 SES 9-030,2,0,0.10,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-03-17,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
432,235599972,2025-03-16 03:00:00,Электробритва Series 9 9510s,528,31,55.92,3,0,0,0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
433,253299215,2025-03-16 03:00:00,Триммер для бороды и усов BT3421,310,20,32.18,3,1,1,3225,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания


In [54]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_final.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [55]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Define headers and project names
projects = {
    'WB-GutenTech': headers_guten,
    'WB-ГиперМаркет': headers_giper,
    'WB-KitchenAid': headers_kitchen,
    'WB-Smart-Market': headers_smart
}

# Calculate yesterday's date
#yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Define the period for the report (only yesterday)
period = {
    "begin": "2025-03-16",
    "end": "2025-03-16"
}

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")

        time.sleep(interval)  # Respect the rate limit

    return all_data

# List to store DataFrames for each project
dataframes = []

# Process each project
for project_name, headers in projects.items():
    # Filter the DataFrame for the current project
    filtered_df = df_final[df_final['Project'] == project_name]
    unique_nmId_values = filtered_df['nmId'].unique().tolist()
    print(f"Total unique nmId values for {project_name}:", len(unique_nmId_values))  # Debugging line

    # Fetch data for all unique nmId values
    all_data = fetch_data_in_batches(unique_nmId_values, period, headers)

    # Flatten the nested 'history' data for easier analysis
    flattened_data = []
    for item in all_data:
        nmID = item['nmID']
        imtName = item['imtName']
        vendorCode = item['vendorCode']
        for history in item['history']:
            history_entry = {
                'nmID': nmID,
                'imtName': imtName,
                'vendorCode': vendorCode,
                'dt': history['dt'],
                'openCardCount': history['openCardCount'],
                'addToCartCount': history['addToCartCount'],
                'addToCartConversion': history['addToCartConversion'],
                'ordersCount': history['ordersCount'],
                'ordersSumRub': history['ordersSumRub'],
                'cartToOrderConversion': history['cartToOrderConversion'],
                'buyoutsCount': history['buyoutsCount'],
                'buyoutsSumRub': history['buyoutsSumRub'],
                'buyoutPercent': history['buyoutPercent']
            }
            flattened_data.append(history_entry)

    # Convert the flattened data to a DataFrame
    df_copy = pd.DataFrame(flattened_data)
    print(f"Data for {project_name}:\n", df_copy.head())

    # Add the DataFrame to the list
    dataframes.append(df_copy)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Total unique nmId values for WB-GutenTech: 187
Data retrieved successfully for batch 1
Data retrieved successfully for batch 2
Data retrieved successfully for batch 3
Data retrieved successfully for batch 4
Data retrieved successfully for batch 5
Data retrieved successfully for batch 6
Data retrieved successfully for batch 7
Data retrieved successfully for batch 8
Data retrieved successfully for batch 9
Data retrieved successfully for batch 10
Data for WB-GutenTech:
        nmID                                            imtName  \
0  64331255  Электрическая зубная щетка взрослая Pro 3 3500...   
1  25088481     Эпилятор женский электрический Silk-epil 9-880   
2  25623718  Бритва мужская электрическая S3 300BT с триммером   
3  52441643         Вакууматор для продуктов с запайкой VR 190   
4  52465014     Вакууматор для продуктов Vacu OneTouch Eco-Set   

       vendorCode          dt  openCardCount  addToCartCount  \
0        Б0052955  2025-03-16             51               2   
1  

In [56]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [57]:
df_copy

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,64331255,Электрическая зубная щетка взрослая Pro 3 3500...,Б0052955,2025-03-16,51,2,4,2,14202,100,0,0,0
1,25088481,Эпилятор женский электрический Silk-epil 9-880,Б0035166,2025-03-16,45,2,4,0,0,0,0,0,0
2,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-03-16,73,4,5,0,0,0,0,0,0
3,52441643,Вакууматор для продуктов с запайкой VR 190,CM-00-00001759,2025-03-16,7,0,0,0,0,0,0,0,0
4,52465014,Вакууматор для продуктов Vacu OneTouch Eco-Set,CM-00-00002210,2025-03-16,23,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,235490798,Эпилятор Silk-epil 5 5-011,7500435225052,2025-03-16,71,3,4,1,9329,33,0,0,0
373,235490802,Эпилятор S9 SES 9-030,7500435225229,2025-03-16,17,3,18,1,12921,33,0,0,0
374,253299215,Триммер для бороды и усов BT3421,4210201447870,2025-03-16,35,3,9,1,3443,33,0,0,0
375,235490795,Эпилятор Silk-epil 7 SE 7-000,7500435225151,2025-03-16,139,10,7,0,0,0,0,0,0


In [58]:
df_unique = df_copy['nmID'].unique()
print(df_unique)

[ 64331255  25088481  25623718  52441643  52465014  25624340  26025671
  43098334 137609254  26026221  52418766 110059376 137609308 135577637
  26053328  46292575  25628178  52469734 137609312  23806943 137609742
 137609754 143429326 137609695 137609782 145266097 143493692 144986300
 137609756 137609323 137609384 138937204 137609370 137609342 137609392
 137609340 143478977 137609389 144980785 137609383 159121849 145813923
 147720621 145266250 159121843 159121846 145266634 159121850 147720619
 159121842 147720616 155383929 155776833 159121836 159121851 147720623
 159121839 159121847 159121837 159121848 162402289 181674546 173292234
 159121859 181893193 159121856 173292233 161420838 169088646 159121853
 173292235 171562370 162402302 159121857 181893195 171562371 173292231
 171562375 159488075 159121852 195507426 200580096 198084188 200481688
 203815244 181893203 187556878 203809741 201854771 181893218 203510173
 200571555 181893209 203505281 200576962 200604181 198208484 181893223
 18840

In [59]:
# Assuming your DataFrame is named df
specific_nmID = 137609342

# Filter the DataFrame for the specific nmID
filtered_df_id = df_copy[df_copy['nmID'] == specific_nmID]

# Group by nmID and sum the ordersSumRub
total_ordersSumRub = filtered_df_id['ordersSumRub'].sum()

print(f"Total ordersSumRub for nmID {specific_nmID}: {total_ordersSumRub}")

Total ordersSumRub for nmID 137609342: 0


In [60]:
df_unique = df_copy['ordersSumRub'].sum()
print(df_unique)

1220850


In [61]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day']).dt.date
df_copy['dt'] = pd.to_datetime(df_copy['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_16060\76939134.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16060\76939134.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,23806943,2025-03-16,Вакууматор для продуктов с запайкой VC 11,415,10,66.21,0,0,0,0,...,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания,0,0,0
1,25088481,2025-03-16,Эпилятор женский электрический Silk-epil 9-880,329,9,74.03,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,2
2,25623718,2025-03-16,Бритва мужская электрическая S3 300BT с триммером,107,5,9.93,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0,0,4
3,25623718,2025-03-16,Бритва мужская электрическая S3 300BT с триммером,628,33,128.74,3,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0,0,4
4,25624340,2025-03-16,Электробритва Series 3 300s,158,10,13.36,3,1,1,5951,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,13792,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,235490801,2025-03-16,Эпилятор Silk-epil 5 5-050,10,1,0.54,0,1,1,6316,...,Wildberries,2100-01-01,2024-10-22,2025-03-17,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,5,39975,18
431,235490802,2025-03-16,Эпилятор S9 SES 9-030,2,0,0.10,0,0,0,0,...,Wildberries,2100-01-01,2024-10-22,2025-03-17,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,12921,3
432,235599972,2025-03-16,Электробритва Series 9 9510s,528,31,55.92,3,0,0,0,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,0,0,4
433,253299215,2025-03-16,Триммер для бороды и усов BT3421,310,20,32.18,3,1,1,3225,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,1,3443,3


In [62]:
df_unique = merged_df_2['ordersSumRub'].sum()
print(df_unique)


1565789


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [64]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
#client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                       int64
day               datetime64[ns]
name_product              object
views                      int64
clicks                     int64
sum                      float64
atbs                       int64
orders                     int64
shks                       int64
sum_price                  int64
advertId                   int64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount                int64
ordersSumRub               int64
addToCartCount             int64
dtype: object
Sample data to insert: [(23806943, Timestamp('2025-03-16 00:00:00'), 'Вакууматор для продуктов с запайкой VC 11', 415, 10, 66.21, 0, 0, 0, 0, 18870595, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), datetime.date(2024, 7, 25